# Instance visualizer
In this notebook I'm going to eplore different ways I can visualize an instant. I'll use this to create a video visualization of continous instances one after the other

#### Some Code (ignoreable)

In [2]:
%matplotlib
# %matplotlib widget

Using matplotlib backend: QtAgg


In [3]:
import dataloader
import pandas as pd
import os
import numpy as np
sid = '08db4255286f'

In [4]:
import importlib
importlib.reload(dataloader)
acc_data = dataloader.acc_data_for_child(sid, True)

loaded dataframe in  6.248 seconds


In [5]:
acc_data.head()

,series_id,step,timestamp,anglez,enmo
0,08db4255286f,0,2018-11-05 14:00:00+00:00,-30.845301,0.0447
1,08db4255286f,1,2018-11-05 14:00:05+00:00,-34.181801,0.0443
2,08db4255286f,2,2018-11-05 14:00:10+00:00,-33.877102,0.0483
3,08db4255286f,3,2018-11-05 14:00:15+00:00,-34.282101,0.0680
4,08db4255286f,4,2018-11-05 14:00:20+00:00,-34.385799,0.0768


In [6]:
events = pd.read_csv(os.path.join('data', 'train_events.csv'))
# events['timestamp'] = pd.to_datetime(events['timestamp'], utc=True)
events[events['series_id']==sid].head()

,series_id,night,event,step,timestamp
398,08db4255286f,1,onset,11988.0,2018-11-06T02:39:00-0400
399,08db4255286f,1,wakeup,14388.0,2018-11-06T05:59:00-0400
400,08db4255286f,2,onset,28680.0,2018-11-07T01:50:00-0400
401,08db4255286f,2,wakeup,31320.0,2018-11-07T05:30:00-0400
402,08db4255286f,3,onset,44184.0,2018-11-07T23:22:00-0400


In [7]:
dataloader.annotate_sid(acc_data, events, sid)

In [8]:
acc_data.sample(10)

,series_id,step,timestamp,anglez,enmo,activity
62873,08db4255286f,62873,2018-11-09 05:19:25+00:00,-37.046700,0.0182,Sleeping
181049,08db4255286f,181049,2018-11-16 01:27:25+00:00,-57.256302,0.0118,Waking
22771,08db4255286f,22771,2018-11-06 21:37:35+00:00,-30.111300,0.0024,Waking
49520,08db4255286f,49520,2018-11-08 10:46:40+00:00,-5.627900,0.1679,Waking
231842,08db4255286f,231842,2018-11-19 00:00:10+00:00,10.583500,0.0086,Waking
354723,08db4255286f,354723,2018-11-26 02:40:15+00:00,-11.386200,0.0215,Waking
1596,08db4255286f,1596,2018-11-05 16:13:00+00:00,87.535004,0.0167,Waking
149481,08db4255286f,149481,2018-11-14 05:36:45+00:00,48.491501,0.0016,Sleeping
97032,08db4255286f,97032,2018-11-11 04:46:00+00:00,-32.855202,0.1020,Waking
366876,08db4255286f,366876,2018-11-26 19:33:00+00:00,-0.996700,0.0890,Waking


##### 20 minutes before and after a timestamp

In [9]:
ts = pd.Timestamp('2018-11-07 05:50:00+00:00')
ts

Timestamp('2018-11-07 05:50:00+0000', tz='UTC')

In [10]:
delta = pd.Timedelta(20, 'min')

In [11]:
(ts - delta, ts + delta)

(Timestamp('2018-11-07 05:30:00+0000', tz='UTC'),
 Timestamp('2018-11-07 06:10:00+0000', tz='UTC'))

In [12]:
idx = (acc_data['timestamp'] >= (ts-delta)) & (acc_data['timestamp'] < (ts+delta))
idx

0         False
1         False
2         False
3         False
4         False
          ...  
440275    False
440276    False
440277    False
440278    False
440279    False
Name: timestamp, Length: 440280, dtype: bool

In [13]:
dat = acc_data.loc[idx]

In [14]:
dat

,series_id,step,timestamp,anglez,enmo,activity
28440,08db4255286f,28440,2018-11-07 05:30:00+00:00,-9.133200,0.0196,Waking
28441,08db4255286f,28441,2018-11-07 05:30:05+00:00,-18.069099,0.0231,Waking
28442,08db4255286f,28442,2018-11-07 05:30:10+00:00,-13.427700,0.0067,Waking
28443,08db4255286f,28443,2018-11-07 05:30:15+00:00,-14.469100,0.0037,Waking
28444,08db4255286f,28444,2018-11-07 05:30:20+00:00,-15.659200,0.0042,Waking
...,...,...,...,...,...,...
28915,08db4255286f,28915,2018-11-07 06:09:35+00:00,6.902200,0.0067,Sleeping
28916,08db4255286f,28916,2018-11-07 06:09:40+00:00,6.904900,0.0067,Sleeping
28917,08db4255286f,28917,2018-11-07 06:09:45+00:00,6.905900,0.0067,Sleeping
28918,08db4255286f,28918,2018-11-07 06:09:50+00:00,6.905900,0.0067,Sleeping


## Visual Frame

To visualize the data at a fixed timestamp I'm taking all the data 20 minutes before and after that timestamp, then plotting their `anglez` and `enmo`. Each point is also given a color based on how many steps it is before or ahead of the timestamp. Finally I added a jitter to the cordinates to reveal points that are hidden one behind the other.

Run the next cell to see an example of a frame where the child transitioned from waking to sleeping

In [15]:
# %matplotlib inline # To show a static image in the notebook
# %matplotlib notebook # To show the interactive video using matplotlib backend
import matplotlib.pyplot as plt
from matplotlib.axes import Axes
import matplotlib.animation as anim
fig, ax = plt.subplots()
ax: Axes = ax
ax.set_xlabel('anglez')
ax.set_ylabel('enmo')
ax.set_title(f'20 minutes around {ts} for {sid}')
x = dat['anglez']
y = dat['enmo']
dat_len = len(dat)
mid = dat['step'].iat[int(dat_len/2)]
c = [i - mid for i in dat['step']]
coll = ax.scatter(x,y, c=c, cmap='plasma', marker='o')
plt.colorbar(coll, ax=ax)
txt = ax.text(-10,0.15,'Some Text')
x_jit = (np.random.random_sample((dat_len,))-0.5)*x.std()
y_jit = (np.random.random_sample((dat_len,))-0.5)*y.std()
jit = [i/100.0 for i in range(30)]
# print(jit)
def animate(frame):
    j = jit[frame]
    coll.set(offsets=[i for i in zip(x+x_jit*j, y+y_jit*j)])
    txt.set_text(f'Jitter factor = {j}')
    return [coll, txt]
fig.tight_layout()
ani = anim.FuncAnimation(func=animate, fig=fig, frames=len(jit), blit=True)
plt.show()

In [211]:
from matplotlib.collections import PathCollection
def init_frame(ax: Axes, data: pd.DataFrame):
    '''
    Create an empty figure with containers for everything that will go in it.

    Returns
    -------
    A list of the artists
    '''
    points = ax.scatter(x=[], y=[], c=[], cmap='plasma')
    ax.set_xlabel('anglez')
    ax.set_ylabel('enmo')
    xmin = data['anglez'].min()
    xmax = data['anglez'].max()
    buff = (xmax-xmin)*0.1
    ax.set_xlim(xmin-buff, xmax+buff)
    ymin = data['enmo'].min()
    ymax = data['enmo'].max()
    buff = (ymax-ymin)*0.1
    ax.set_ylim(ymin-buff, ymax+buff)
    activity_text = ax.text((xmax+xmin)/2, ymax-data['enmo'].std(), "SOME TEXT", ha='center', weight='bold', va='bottom')
    return [points, activity_text]

def draw_frame(ts: pd.Timestamp, data: pd.DataFrame, window: pd.Timedelta, x_jit_scale: int, y_jit_scale: int, points: PathCollection, activity_text: plt.Text):
    artists = []
    if type(ts) is not pd.Timestamp:
        ts = pd.Timestamp(ts)
    idx = (data['timestamp'] >= ts-window) & (data['timestamp'] < ts+window)
    data = data.loc[idx, ['step', 'anglez', 'enmo', 'activity']]
    # Set X, Y points
    x = data['anglez']
    y = data['enmo']
    dat_len = len(data)
    mid = dat['step'].iat[int(dat_len/2)]
    c = [i - mid for i in dat['step']]
    x_jit = (np.random.random_sample(x.shape)-0.5)*x_jit_scale
    y_jit = (np.random.random_sample(y.shape)-0.5)*y_jit_scale
    jit = 0.3
    points.set(offsets=list(zip(x+x_jit*jit,y+y_jit*jit)), array=c)
    artists.append(points)
    # Set the text box
    txt = f"{ts}\n{data['activity'].iat[0]} => {data['activity'].iat[-1]}"
    activity_text.set_text(txt)
    artists.append(activity_text)
    # if txt != activity_text.get_text():
        # activity_text.set_text(txt)
    #     artists.append(activity_text)
    return artists

In [214]:
import tqdm

fig, ax = plt.subplots()
artists = init_frame(ax, acc_data)
ani = anim.FuncAnimation(
    fig=fig, 
    func=draw_frame, 
    frames=tqdm.notebook.tqdm(acc_data['timestamp'][::60]), 
    fargs=(acc_data, pd.Timedelta(20, 'min'), acc_data['anglez'].std(), 2*acc_data['enmo'].std(), *artists),
    blit=True,
    interval=100
)
# plt.show()

  0%|          | 0/7338 [00:00<?, ?it/s]

In [190]:
ani.save?

Signature:
ani.save(
    filename,
    writer=None,
    fps=None,
    dpi=None,
    codec=None,
    bitrate=None,
    extra_args=None,
    metadata=None,
    extra_anim=None,
    savefig_kwargs=None,
    *,
    progress_callback=None,
)
Docstring:
Save the animation as a movie file by drawing every frame.

Parameters
----------
filename : str
    The output filename, e.g., :file:`mymovie.mp4`.

writer : `MovieWriter` or str, default: :rc:`animation.writer`
    A `MovieWriter` instance to use or a key that identifies a
    class to use, such as 'ffmpeg'.

fps : int, optional
    Movie frame rate (per second).  If not set, the frame rate from the
    animation's frame interval.

dpi : float, default: :rc:`savefig.dpi`
    Controls the dots per inch for the movie frames.  Together with
    the figure's size in inches, this controls the size of the movie.

codec : str, default: :rc:`animation.codec`.
    The video codec to use.  Not all codecs are supported by a given
    `MovieWriter`.

b

In [215]:
ani.save(os.path.join('vids', 'activity3.mp4'))